# NNDL Project
Yihan Chen(yc4170)

In [ ]:
import os
import gc
import cv2
import copy
import time
import torch
import random
import string
import joblib
import tifffile
import numpy as np 
import pandas as pd 
import torch.nn as nn
import seaborn as sns
from PIL import Image
from random import randint
from einops import rearrange
from torchvision import models
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from torch.optim import lr_scheduler
from einops.layers.torch import Rearrange
from efficientnet_pytorch import EfficientNet
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import warnings; warnings.filterwarnings("ignore")
import torchvision.transforms as T
import torchvision.transforms.functional as F
import timm
from pprint import pprint
gc.enable()

In [ ]:
# install packages for vm
# ! pip install timm
# ! pip install opencv-python
# ! pip install joblib
# ! pip install tifffile
# ! pip install pandas
# ! pip install seaborn
# ! pip install einops
# ! pip install tqdm
# ! pip install efficientnet_pytorch
# ! pip install sklearn

In [ ]:
# Unzip image files

# import zipfile
# with zipfile.ZipFile("train_shuffle.zip", 'r') as zip_ref:
#     zip_ref.extractall("")
# with zipfile.ZipFile("test_shuffle.zip", 'r') as zip_ref:
#     zip_ref.extractall("")

## Data Preparation

In [ ]:
# Read data
train_df = pd.read_csv("train_data.csv")
test_df = pd.read_csv("example_test_submission.csv")

super_mapping = pd.read_csv("super_classes_mapping.csv")
sub_mapping = pd.read_csv("sub_classes_mapping.csv")

In [ ]:
# Ser seed for reproducing
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

seed = 42
seed_everything(seed)
debug = False
# generate_new = False

In [ ]:
train_df = train_df.rename({'superclass_index':'label', 'subclass_index':'label2'}, axis='columns')
train_df

### Create New image (for predicting new)

In [ ]:
# DO NOT RUN SECOND TIME

# spectrum = [0, 64, 128, 192, 255]
# SIZE = 8
# index = 6472 #start from 6472.jpg

# # Pure -- total 125 images
# for r in spectrum:
#   for g in spectrum:
#     for b in spectrum:
#       img = Image.new('RGB', (SIZE, SIZE), color = (r,g,b))
#       img.save('train_shuffle/' + str(index) + '.jpg')
#       index += 1

# # Random -- total 150 images
#     # colorful -- 50 imagess
#     # grey -- 50 images
#     # black&white -- 50 images
# for i in range(50):
#   imarray = np.random.rand(SIZE,SIZE,3) * 255
#   im = Image.fromarray(imarray.astype('uint8')).convert('RGB')
#   im.save('train_shuffle/' + str(index) + '.jpg')
#   index += 1
#   im2 = im.convert("L")
#   im2.save('train_shuffle/' + str(index) + '.jpg')
#   index += 1
#   im3 = im2.convert("1")
#   im3.save('train_shuffle/' + str(index) + '.jpg')
#   index += 1

In [ ]:
train_df_sub = copy.deepcopy(train_df)
for i in range(6472,6747):
    train_df_sub.loc[i] = {'image': str(i) + '.jpg', 'label': 3, 'label2': 89}

In [ ]:
# Check number of images
_, _, files = next(os.walk("/home/ecbm4040/train_shuffle"))
file_count = len(files)
file_count

In [ ]:
# Check train_df & train_df_sub
train_df, train_df_sub

### Read image

In [ ]:
class ImgDataset(Dataset):
    def __init__(self, df):
        self.df = df 
        self.train = 'label' in df.columns    

    def __len__(self): return len(self.df)    
    def __getitem__(self, index):
        paths = ["test_shuffle/", "train_shuffle/"]

        image = cv2.imread(paths[self.train] + self.df.iloc[index].image)
        
        # add transform
        image = Image.fromarray(image)
#         image = F.adjust_contrast(image, 1.2)
#         image = F.adjust_brightness(image, 1.1)
        image = F.adjust_sharpness(image, 1.1)
        image = np.array(image)
        
        image = cv2.resize(image, (512, 512)).transpose(2, 0, 1)

        label = None
        if self.train:
          label = self.df.iloc[index].label

        return image, label

## Training

In [ ]:
def train_model(model, dataloaders_dict, criterion, optimizer, num_epochs):
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        model.cuda()       
        for phase in ['train', 'val']:
            if phase == 'train': model.train()
            else: model.eval()
               
            epoch_loss = 0.0
            epoch_acc = 0
            
            dataloader = dataloaders_dict[phase]
            for item in tqdm(dataloader, leave=False):
                images = item[0].cuda().float()
                classes = item[1].cuda().long()
                optimizer.zero_grad()                
                with torch.set_grad_enabled(phase == 'train'):
                    output = model(images)
                    loss = criterion(output, classes)
                    _, preds = torch.max(output, 1)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    epoch_loss += loss.item() * len(output)
                    epoch_acc += torch.sum(preds == classes.data)                    
            data_size = len(dataloader.dataset)
            epoch_loss = epoch_loss / data_size
            epoch_acc = epoch_acc.double() / data_size
            print(f'Epoch {epoch + 1}/{num_epochs} | {phase:^5} | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f}')    
        if epoch_acc > best_acc:
            torch.save(model.state_dict(), "model_state.pth")
#             traced = torch.jit.trace(model.cpu(), torch.rand(1, 3, 512, 512))
#             traced.save('model.pth')
            best_acc = epoch_acc
    print()
    print('Best val Acc: {:4f}'.format(best_acc))

In [ ]:
# model_names = timm.list_models(pretrained=True)
# pprint(model_names)

In [ ]:
os.environ['WANDB_CONSOLE'] = 'off'

## XCiT - Super Category

In [ ]:
# xcit_nano_12_p16_384_dist Epoch 1/1 |  val  | Loss: 3.8789 | Acc: 0.5200

model = timm.create_model('xcit_nano_12_p16_384_dist')
train, val = train_test_split(train_df, test_size=0.2, random_state=42, stratify = train_df.label)
test_0, val = train_test_split(val, test_size=0.5, random_state=42, stratify = val.label)


batch_size = 64
train_loader = DataLoader(ImgDataset(train), batch_size=batch_size, shuffle=False, num_workers=1)
val_loader = DataLoader(ImgDataset(val), batch_size=batch_size, shuffle=False, num_workers=1)

dataloaders_dict = {"train": train_loader, "val": val_loader}
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4/2)
# weight_decay (float, optional) – weight decay coefficient (default: 1e-2)
train_model(model, dataloaders_dict, criterion, optimizer, 15)

In [ ]:
test_df

In [ ]:
class ImgDataset_test(Dataset):
    def __init__(self, df):
        self.df = df 
        self.train = 'label' in df.columns    

    def __len__(self): return len(self.df)    
    def __getitem__(self, index):
        paths = ["test_shuffle/", "train_shuffle/"]

        image = cv2.imread(paths[self.train] + self.df.iloc[index].image_index)
        
        # add transform
        image = Image.fromarray(image)
#         image = F.adjust_contrast(image, 1.2)
#         image = F.adjust_brightness(image, 1.1)
        image = F.adjust_sharpness(image, 1.1)
        image = np.array(image)
        
        image = cv2.resize(image, (512, 512)).transpose(2, 0, 1)

        label = 0 # useless
        if self.train:
          label = self.df.iloc[index].label
        
        image_index = self.df.iloc[index].image_index
        
        return image, image_index

In [ ]:
def predict(model, dataloader):
    model.cuda()
    model.eval()
    dataloader = dataloader
    outputs = []
    s = nn.Softmax(dim=1)
    ids = []
    
    for item in tqdm(dataloader, leave=False):
        image_index = item[1][0]
#         try:
        images = item[0].cuda().float()
        ids.append(image_index)
        output = model(images)
        outputs.append(s(output.cpu()[:,:3])[0].detach().numpy())
        
#         except:
#             ids.append(image_index)
#             outputs.append(s(torch.tensor([[1, 1]]).float())[0].detach().numpy())
    return np.array(outputs), ids    

In [ ]:
# model = torch.jit.load('model.pth')
model = timm.create_model('xcit_nano_12_p16_384_dist')
model.load_state_dict(torch.load("model_state.pth"))

batch_size = 1
test_loader = DataLoader(
    ImgDataset(test_0), 
    batch_size = batch_size, 
    shuffle = False, 
    num_workers = 1
)
test_results_0, ids_0 = predict(model, test_loader)
sum(test_results_0.argmax(1) == test_0.label)/len(test_0)

In [ ]:
batch_size = 1
test_loader = DataLoader(
    ImgDataset_test(test_df), 
    batch_size = batch_size, 
    shuffle = False, 
    num_workers = 1
)

anss, ids = predict(model, test_loader)
res = pd.DataFrame({"image_index" : ids, "superclass_index" : anss.argmax(1)})
res

In [ ]:
res.to_csv('super_79134.csv', index=False)

## XCiT - Sub Category

In [ ]:
train_df_sub = train_df_sub.rename({'label':'label2', 'label2':'label'}, axis='columns')
train_df_sub

In [ ]:
def predict(model, dataloader):
    model.cuda()
    model.eval()
    dataloader = dataloader
    outputs = []
    s = nn.Softmax(dim=1)
    ids = []
    
    for item in tqdm(dataloader, leave=False):
        image_index = item[1][0]
#         try:
        images = item[0].cuda().float()
        ids.append(image_index)
        output = model(images)
        outputs.append(s(output.cpu()[:,:90])[0].detach().numpy())
        
#         except:
#             ids.append(image_index)
#             outputs.append(s(torch.tensor([[1, 1]]).float())[0].detach().numpy())
    return np.array(outputs), ids 

def train_model(model, dataloaders_dict, criterion, optimizer, num_epochs):
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        model.cuda()       
        for phase in ['train', 'val']:
            if phase == 'train': model.train()
            else: model.eval()
               
            epoch_loss = 0.0
            epoch_acc = 0
            
            dataloader = dataloaders_dict[phase]
            for item in tqdm(dataloader, leave=False):
                images = item[0].cuda().float()
                classes = item[1].cuda().long()
                optimizer.zero_grad()                
                with torch.set_grad_enabled(phase == 'train'):
                    output = model(images)
                    loss = criterion(output, classes)
                    _, preds = torch.max(output, 1)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    epoch_loss += loss.item() * len(output)
                    epoch_acc += torch.sum(preds == classes.data)                    
            data_size = len(dataloader.dataset)
            epoch_loss = epoch_loss / data_size
            epoch_acc = epoch_acc.double() / data_size
            print(f'Epoch {epoch + 1}/{num_epochs} | {phase:^5} | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f}')    
        if epoch_acc > best_acc:
            torch.save(model.state_dict(), "model_state_sub.pth")
#             traced = torch.jit.trace(model.cpu(), torch.rand(1, 3, 512, 512))
#             traced.save('model.pth')
            best_acc = epoch_acc
    print()
    print('Best val Acc: {:4f}'.format(best_acc))

In [ ]:
# xcit_nano_12_p16_384_dist Epoch 1/1 |  val  | Loss: 3.8789 | Acc: 0.5200

model = timm.create_model('xcit_nano_12_p16_384_dist')
train, val = train_test_split(train_df_sub, test_size=0.2, random_state=42, stratify = train_df_sub.label)
test_0, val = train_test_split(val, test_size=0.5, random_state=42, stratify = val.label)

batch_size = 64
train_loader = DataLoader(ImgDataset(train), batch_size=batch_size, shuffle=False, num_workers=1)
val_loader = DataLoader(ImgDataset(val), batch_size=batch_size, shuffle=False, num_workers=1)

dataloaders_dict = {"train": train_loader, "val": val_loader}
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4/2) 
# weight_decay (float, optional) – weight decay coefficient (default: 1e-2)
sub_model = train_model(model, dataloaders_dict, criterion, optimizer, 25)

In [ ]:
model = timm.create_model('xcit_nano_12_p16_384_dist')
model.load_state_dict(torch.load("model_state_sub.pth"))

batch_size = 1
test_loader = DataLoader(
    ImgDataset(test_0), 
    batch_size = batch_size, 
    shuffle = False, 
    num_workers = 1
)
test_results_0, ids_0 = predict(model, test_loader)
sum(test_results_0.argmax(1) == test_0.label)/len(test_0)

In [ ]:
batch_size = 1
test_loader = DataLoader(
    ImgDataset_test(test_df), 
    batch_size = batch_size, 
    shuffle = False, 
    num_workers = 1
)

anss, ids = predict(model, test_loader)
res = pd.DataFrame({"image_index" : ids, "subclass_index" : anss.argmax(1)})
# res['subclass_index'] = anss.argmax(1)
res

In [ ]:
res.to_csv('sub_2074.csv', index=False)

In [ ]:
# clear memory
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Early Stop 
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [ ]:
early_stopper = EarlyStopper(patience=3, min_delta=10)
for epoch in np.arange(n_epochs):
    train_loss = train_one_epoch(model, train_loader)
    validation_loss = validate_one_epoch(model, validation_loader)
    if early_stopper.early_stop(validation_loss):             
        break